# ALU CE Bridge — Live Telemetry Monitor
Reads all game telemetry from the CE bridge file and displays in real-time:
**Race Timer**, **Progress**, **Visual Timer** (race state), **Gear**, **RPM**, **Checkpoint**

In [ ]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from src.modules.ce_client import CheatEngineClient

client = CheatEngineClient(poll_interval=0.002)  # 2ms = 500Hz polling
client.start()

In [ ]:
import time
from IPython.display import display, HTML, clear_output

def format_timer_us(timer_us):
    """Format microseconds as MM:SS.mmm"""
    if timer_us <= 0:
        return "00:00.000"
    total_ms = timer_us // 1000   # µs → ms
    minutes = total_ms // 60000
    seconds = (total_ms % 60000) // 1000
    ms = total_ms % 1000
    return f"{minutes:02d}:{seconds:02d}.{ms:03d}"

RACE_ENDED_THRESHOLD_US = 3250  # µs gap before we confirm race ended

# State tracking across polls
prev_checkpoint = 0
estimated_finish_us = None  # captured once at finish detection

def detect_race_state(visual_timer, timer_raw, progress, gear):
    """
    Detect current game state from telemetry values.

    Returns: (label, emoji, color)
    """
    if visual_timer == 1_000_000:
        return "In Menus", "🏠", "#89dceb"
    elif visual_timer == 0:
        if gear == 0:
            return "Race Starting", "🏁", "#f9e2af"
        else:
            return "In Menus", "🏠", "#89dceb"
    elif timer_raw > 0 and (visual_timer - timer_raw) > RACE_ENDED_THRESHOLD_US:
        return "Race Ended", "🏆", "#f5c211"
    else:
        return "Race In Progress", "🏎️", "#a6e3a1"

def check_finish_trigger(checkpoint, prev_cp, progress, timer_raw):
    """
    Detect the instant the race finishes:
      progress > 99% AND (checkpoint increased OR checkpoint wrapped to 0)

    Returns timer_raw at that instant, or None if not triggered.
    """
    if progress > 99.0:
        cp_increased = checkpoint > prev_cp
        cp_wrapped = (checkpoint == 0 and prev_cp != 0)
        if cp_increased or cp_wrapped:
            return timer_raw
    return None

print("Live telemetry monitor running — interrupt kernel to stop\n")

try:
    while True:
        vals = client.get_all_values()
        stats = client.get_stats()
        connected = stats['ce_connected']

        timer_raw = vals['timer_raw']
        progress = vals['progress']
        rpm = vals['rpm']
        gear = vals['gear']
        checkpoint = vals['checkpoint']
        visual_timer = vals['visual_timer']

        pct = round(progress * 100, 2) if 0.0 <= progress <= 1.0 else round(progress, 2)
        timer_str = format_timer_us(timer_raw)

        # Gear display
        gear_display = "N" if gear == 0 else str(gear)

        # Race state detection
        state_label, state_emoji, state_color = detect_race_state(visual_timer, timer_raw, progress, gear)

        # Finish detection — capture timer at the exact poll it triggers
        finish = check_finish_trigger(checkpoint, prev_checkpoint, pct, timer_raw)
        if finish is not None:
            estimated_finish_us = finish
        prev_checkpoint = checkpoint

        # Reset estimated finish when back in menus or new race starting
        if state_label in ("In Menus", "Race Starting"):
            estimated_finish_us = None

        # Format estimated finish
        if estimated_finish_us is not None:
            est_str = format_timer_us(estimated_finish_us)
            est_raw = estimated_finish_us
            est_html = f"""
            <div style="display: flex; align-items: baseline; gap: 12px; margin-bottom: 4px;">
                <span style="font-size: 13px; color: #6c7086; min-width: 90px;">EST. FINISH</span>
                <span style="font-size: 28px; font-weight: bold; color: #f5c211;">{est_str}</span>
            </div>
            <div style="font-size: 12px; color: #45475a; margin-bottom: 16px; padding-left: 102px;">
                Raw: {est_raw} µs
            </div>"""
        else:
            est_html = ""

        clear_output(wait=True)
        display(HTML(f"""
        <div style="font-family: 'Cascadia Code', 'Consolas', monospace; padding: 16px;
                    background: #1e1e2e; color: #cdd6f4; border-radius: 10px;
                    border: 1px solid #313244;">

            <!-- Status Bar -->
            <div style="display: flex; justify-content: space-between; align-items: center;
                        font-size: 13px; color: #6c7086; margin-bottom: 8px;">
                <span style="color: {'#a6e3a1' if connected else '#f38ba8'}; font-weight: bold;">
                    ● {'Connected' if connected else 'Disconnected'}
                </span>
                <span>
                    Reads: {stats['ce_reads_ok']} &nbsp;|&nbsp; Errors: {stats['ce_reads_failed']}
                </span>
            </div>
            <hr style="border: none; border-top: 1px solid #313244; margin: 8px 0 12px 0;">

            <!-- Race State -->
            <div style="display: flex; align-items: center; gap: 10px; margin-bottom: 4px;">
                <span style="font-size: 13px; color: #6c7086; min-width: 90px;">RACE STATE</span>
                <span style="font-size: 24px;">{state_emoji}</span>
                <span style="font-size: 22px; font-weight: bold; color: {state_color};">{state_label}</span>
            </div>
            <div style="font-size: 12px; color: #45475a; margin-bottom: 16px; padding-left: 102px;">
                Raw: {visual_timer}
            </div>

            <hr style="border: none; border-top: 1px solid #313244; margin: 0 0 12px 0;">

            <!-- Race Timer -->
            <div style="display: flex; align-items: baseline; gap: 12px; margin-bottom: 4px;">
                <span style="font-size: 13px; color: #6c7086; min-width: 90px;">RACE TIME</span>
                <span style="font-size: 36px; font-weight: bold; color: #89b4fa;">{timer_str}</span>
            </div>
            <div style="font-size: 12px; color: #45475a; margin-bottom: 16px; padding-left: 102px;">
                Raw: {timer_raw} µs
            </div>

            <!-- Estimated Finish (shown only after finish detected) -->
            {est_html}

            <!-- Progress -->
            <div style="display: flex; align-items: baseline; gap: 12px; margin-bottom: 6px;">
                <span style="font-size: 13px; color: #6c7086; min-width: 90px;">PROGRESS</span>
                <span style="font-size: 28px; font-weight: bold; color: #f9e2af;">{pct:.2f}%</span>
            </div>
            <div style="margin: 0 0 16px 102px; background: #313244; border-radius: 4px;
                        height: 14px; overflow: hidden;">
                <div style="background: linear-gradient(90deg, #a6e3a1, #94e2d5);
                            height: 100%; border-radius: 4px;
                            width: {min(pct, 100):.1f}%; transition: width 0.15s ease;">
                </div>
            </div>

            <hr style="border: none; border-top: 1px solid #313244; margin: 0 0 12px 0;">

            <!-- Gear + RPM + Checkpoint Row -->
            <div style="display: flex; gap: 24px;">
                <!-- Gear -->
                <div style="flex: 1; background: #313244; border-radius: 8px; padding: 12px;
                            text-align: center;">
                    <div style="font-size: 11px; color: #6c7086; text-transform: uppercase;
                                letter-spacing: 1px; margin-bottom: 4px;">Gear</div>
                    <div style="font-size: 32px; font-weight: bold; color: #fab387;">
                        {gear_display}
                    </div>
                </div>
                <!-- RPM -->
                <div style="flex: 2; background: #313244; border-radius: 8px; padding: 12px;
                            text-align: center;">
                    <div style="font-size: 11px; color: #6c7086; text-transform: uppercase;
                                letter-spacing: 1px; margin-bottom: 4px;">RPM</div>
                    <div style="font-size: 32px; font-weight: bold; color: #f38ba8;">
                        {rpm:,}
                    </div>
                </div>
                <!-- Checkpoint -->
                <div style="flex: 1; background: #313244; border-radius: 8px; padding: 12px;
                            text-align: center;">
                    <div style="font-size: 11px; color: #6c7086; text-transform: uppercase;
                                letter-spacing: 1px; margin-bottom: 4px;">Checkpoint</div>
                    <div style="font-size: 32px; font-weight: bold; color: #94e2d5;">
                        {checkpoint}
                    </div>
                </div>
            </div>
        </div>
        """))

        time.sleep(0.05)  # 20 Hz display refresh

except KeyboardInterrupt:
    print("Stopped.")

In [ ]:
# Run this cell to cleanly stop the client
client.stop()